# Modelo de recomendacion

### Cargo el dataset usado para el modelo

In [ ]:
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split as train_test_splitSV
from sklearn.preprocessing import LabelEncoder
import pickle
from surprise import SVD
from surprise.model_selection import GridSearchCV
from surprise import accuracy
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
user_reviews = pd.read_csv('../datasets/user_reviews.csv',usecols=['user_id','item_id','sentiment_analysis','recommend'])
label_encoder = LabelEncoder()
#elimino la columna user_id
user_reviews['user_id_num'] = label_encoder.fit_transform(user_reviews['user_id'])



### Voy a generar un rating a partir de recommend y sentiment_analysis para que este entre 0 y 5

In [ ]:
user_reviews['rating'] = np.where(user_reviews['recommend'] == True,  # Si 'recommend' es True
                                 np.where(user_reviews['sentiment_analysis'] == 2, 5,  # Si 'sentimiento' es positivo
                                          np.where(user_reviews['sentiment_analysis'] == 1, 3,  # Si 'sentimiento' es neutro
                                                   1)),  # Si 'sentimiento' es negativo cuando 'recommend' es True
                                 np.where(user_reviews['sentiment_analysis'] == 2, 4,  # Si 'sentimiento' es positivo
                                          np.where(user_reviews['sentiment_analysis'] == 1, 2,  # Si 'sentimiento' es neutro
                                                   0)))  # Si 'sentimiento' es negativo cuando 'recommend' es False

In [ ]:
user_reviews.sample(5)

,user_id,item_id,recommend,sentiment_analysis,user_id_num,rating
50605,76561198063055125,239070,True,2,5348,5
53520,76561198076114293,17460,True,2,7647,5
46727,newhollandarmy,223470,True,1,22668,3
46078,76561198024028497,227100,False,0,1647,0
53714,TheMarshmallowMan101,730,True,2,17361,5


In [ ]:
print(user_reviews['rating'].max())
print(user_reviews['rating'].min())

5
0


### Genero id numericos para user_id usando label encoder

In [ ]:
reader = Reader(rating_scale=(0,5))
data = Dataset.load_from_df(user_reviews[['user_id_num','item_id','rating']], reader)


### Separo el dataset en entrenamiento y testeo

In [ ]:
train,test = train_test_splitSV(data,test_size = .25)

## Voy a usar un modelo De descomposicion en valor singular (SVD), el cual es un filtro colaborativo
### Voy a usar los datos de 
* user_id
* item_id
* sentiment_analysis: como rating

### Aplico GridSearch para encontrar al modelo con los mejores hiperparametros

In [ ]:
param_grid = {'n_factors': [5,50,100],'n_epochs': [5, 10,20], 'lr_all': [0.001, 0.002, 0.005],
              'reg_all': [0.002, 0.02, 0.2]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, n_jobs = -1)
gs.fit(data)


In [ ]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])
best_model = gs.best_estimator['rmse']

1.5840100111706839
{'n_factors': 100, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.2}


### Veo que el modelo tiene un rmse de 1.06 lo cual esta cerca de 1lo cual en la escala esta bien
### Ademas veo los mejores hiperparametros

## Por ultimo defino el modelo con los mejores hiperparametros y lo exporto

In [ ]:
model = SVD(n_factors=100,n_epochs=20,lr_all=0.005,reg_all=0.2)
model.fit(train)

In [ ]:
from surprise import accuracy
train_predictions = model.test(train.build_testset())
test_predictions = model.test(test)
print(f"RMSE en el conjunto de entrenamiento: {accuracy.rmse(train_predictions)}")
print(f"RMSE en el conjunto de prueba: {accuracy.rmse(test_predictions)}")

RMSE: 1.2764
RMSE en el conjunto de entrenamiento: 1.2764039278981176
RMSE: 1.5817
RMSE en el conjunto de prueba: 1.5817043122021668


### EL modelo obtuvo un mejor rendimiento en los datos de testeo el rmse 1.5 me indica que en en el ranking de 0 5 lo que indica una desviacion de 1.5 si bien tiene mucho rango de mejora para las columnas usadas como rating esta bastante bien

In [ ]:
with open('./SVD_model.pkl', 'wb') as file: # Exporto mi modelo
    pickle.dump(model, file)

In [ ]:
print(model.predict(4124,12354))
print(model.predict(4124,1244))
print(model.predict(4124,154))
print(model.predict(4124,12554))
print(model.predict(4124,1213))
print(model.predict(4124,1884))
print(model.predict(4124,35354))
print(model.predict(4124,1324))
print(model.predict(4124,13224))


user: 4124       item: 12354      r_ui = None   est = 3.82   {'was_impossible': False}
user: 4124       item: 1244       r_ui = None   est = 3.82   {'was_impossible': False}
user: 4124       item: 154        r_ui = None   est = 3.82   {'was_impossible': False}
user: 4124       item: 12554      r_ui = None   est = 3.82   {'was_impossible': False}
user: 4124       item: 1213       r_ui = None   est = 3.82   {'was_impossible': False}
user: 4124       item: 1884       r_ui = None   est = 3.82   {'was_impossible': False}
user: 4124       item: 35354      r_ui = None   est = 3.82   {'was_impossible': False}
user: 4124       item: 1324       r_ui = None   est = 3.82   {'was_impossible': False}
user: 4124       item: 13224      r_ui = None   est = 3.82   {'was_impossible': False}


## A contuacion hare un modelo basado en 
* Para esto usare la tabla steam_games y los generos de los juegos

In [ ]:
def item_rec(app_name:int):
    generos = list(steam_games.drop(columns=['app_name','price','id','developer','Accounting','Year']).columns)
    generos
    perfiles_items = []
    for _, row in steam_games.iterrows():
        perfil_item = []
        for genero in generos:
            perfil_item.append(row[genero])
        perfiles_items.append(perfil_item)

    perfiles_items_df = pd.DataFrame(perfiles_items, columns=generos)
    perfiles_items_df['app_name'] = steam_games['app_name']
    perfiles_items_df['id'] = steam_games['id']

    # Nombre de la aplicación para la cual deseas obtener recomendaciones
    if app_name not in list(steam_games['id']):
        return 'EL item no se a podido encontrar'

    # Encuentra el índice correspondiente al nombre de la aplicación
    app_index = perfiles_items_df[perfiles_items_df['id'] == app_name].index[0]
    perfiles_items_array = perfiles_items_df.drop(columns=['app_name','id']).values
    # Obtén el perfil de la aplicación del DataFrame
    app_profile = perfiles_items_array[app_index].reshape(1, -1)

    # Calcula la similitud de coseno entre el perfil de la aplicación y todos los perfiles de items
    similarity_scores = cosine_similarity(app_profile, perfiles_items_array)

    # Ordena los juegos por similitud y toma los primeros 5 juegos recomendados
    recommended_games = np.argsort(similarity_scores[0])[::-1][:5]

    # Obtiene los índices de los juegos recomendados en el DataFrame original
    recommended_game_indices = perfiles_items_df.index[recommended_games]

    # Obtiene los nombres de los juegos recomendados
    recommended_game_names = perfiles_items_df.loc[recommended_game_indices, 'app_name']
    return {
        'Recomendacion 1':recommended_game_names.iloc[0],
        'Recomendacion 2':recommended_game_names.iloc[1],
        'Recomendacion 3':recommended_game_names.iloc[2],
        'Recomendacion 4':recommended_game_names.iloc[3],
        'Recomendacion 5':recommended_game_names.iloc[4]
            }


In [ ]:
item_rec(658870)

{'Recomendacion 1': 'EXIT 2 - Directions',
 'Recomendacion 2': 'Keep Rollin!',
 'Recomendacion 3': 'Tetropunk',
 'Recomendacion 4': "Alexa's Wild Night",
 'Recomendacion 5': 'Zup! 7 - 4:3 Pack'}